<a href="https://colab.research.google.com/github/cape2021/Team2/blob/main/Generate_Video_Stylegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Requirements and Preparation**

In [ ]:
#Use of Tensor flow 1.X is recomended for the current notebook
%tensorflow_version 1.x #If you are using Google Colab

In [ ]:
%pip install Google-Colab-Transfer #If you are Using Google Colab 

**Mount Google Drive** 

In case you are using google cloud to train

In [ ]:
import colab_transfer

gd = colab_transfer.get_path_to_home_of_google_drive()
lm = colab_transfer.get_path_to_home_of_local_machine()

colab_transfer.mount_google_drive()

**Clone Stylegan2-ada  Repository**

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada.git


**Generate Images**

In [ ]:
# reference: https://arxiv.org/pdf/2009.05673.pdf

import sys
import pickle
import os
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt

sys.path.insert(0, "/content/stylegan2-ada")

import dnnlib
import dnnlib.tflib as tflib

def seed2vec(Gs, seed):
  rnd = np.random.RandomState(seed)
  return rnd.randn(1, *Gs.input_shape[1:])

def init_random_state(Gs, seed):
  rnd = np.random.RandomState(seed) 
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]

def display_image(image):
  plt.axis('off')
  plt.imshow(image)
  plt.show()

def generate_image(Gs, z, truncation_psi):
    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    label = np.zeros([1] + Gs.input_shapes[1][1:])
    images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
    return images[0]

In [ ]:
import IPython.display

tflib.init_tf()

#add your pickle
with open("/content/drive/MyDrive/PEIDATASET/peidata5OUT/00067-peidata5-auto_no_ramp1-kimg30000-p0.713-resumecustom-freezed10/network-snapshot-029567.pkl", "rb") as fp:
  _G, _D, Gs = pickle.load(fp)

In [ ]:
# Choose your own starting and ending seed.
SEED_FROM = 2000
SEED_TO = 2200

# Generate the images for the seeds.
for i in range(SEED_FROM, SEED_TO):
  print(f"Seed {i}")
  init_random_state(Gs, 10)
  z = seed2vec(Gs, i)
  img = generate_image(Gs, z, 1.0)
  display_image(img)

In [ ]:
# Choose your seeds to morph through and the number of steps to take to get to each.
SEEDS = [2182,2180]
import random
random.shuffle(SEEDS)
SEEDS.append(SEEDS[0])
STEPS = 50

# Remove any prior results
!rm /content/results/* 

from tqdm.notebook import tqdm

os.makedirs("./results/", exist_ok=True)

# Generate the images for the video.
idx = 0
for i in range(len(SEEDS)-1):
  v1 = seed2vec(Gs, SEEDS[i])
  v2 = seed2vec(Gs, SEEDS[i+1])

  diff = v2 - v1
  step = diff / STEPS
  current = v1.copy()

  for j in tqdm(range(STEPS), desc=f"Seed {SEEDS[i]}"):
    current = current + step
    init_random_state(Gs, 10)
    img = generate_image(Gs, current, 1.0)
    PIL.Image.fromarray(img, 'RGB').save(f'./results/frame-{idx}.png')
    idx+=1
 
# Link the images into a video.
!ffmpeg -r 30 -i /content/results/frame-%d.png -vcodec mpeg4 -y movie.mp4